# Coefficient Investigation using the Logistic Regression algorithm

This experiment ran on the question feature. It consists of some preprocessing steps that are mentioned below and of course the main process of applying the Logistic Regression algorithm

In [6]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

df = pd.read_csv("UniqueQuestions.csv")
df1 = df["Question"]

print "Here there are some sample questions from the question feature:"
print
print df1[0:5]

Here there are some sample questions from the question feature:

0    Can you see more than 50% of a word on the per...
1                             Is the persons' hair up?
2                           Is the person's hair dyed?
3                           Is the person's hair dyed?
4           is the girl wearing anything on their head
Name: Question, dtype: object


## Creation of the DFCoef database

Further information are provided in the thesis doc. This database, in general was created to support the experiment and run the algorithm which requires some binary values. These values were captured after manually checking, each of the FOUR words (eyes, wear, skin and hair) in the content of the question feature. The below is just an example of the "hair" word.

In [7]:
sentences = []

for items in range(len(df["Question"])):
    line = str(df["Question"][items])
    if "hair" in line:
        #df["Blonde"][items] == 1
        sentences.append(1)
    else:
        #df["Blonde"][items] == 0
        sentences.append(0)
        
df["hair"] = sentences


## Preprocessing

In [8]:
allsentences = []

for items in df["Question"]:
    items = str(items)
    sentence = []
    for words in items.split():
        #print words
        pattern = re.search(r'(.*)(\[comma])',words)
        if pattern:
            sentence.append(pattern.group(1).lower())
        else:
            sentence.append(words.lower())
    allsentences.append(sentence)


In [9]:
lista=  []

for items in allsentences:
    captured = " ".join(items)
    lista.append(captured)

In [10]:
listb = []

for items in lista:
    summing = []
    items = items.split()
    for words in items:
        words = words.strip("\',?.")
        #print words
        if words not in stopwords.words("english"):
            summing.append(words)
    listb.append(summing)
    
    

In [11]:
liste = []

for items in listb:
    temporar = []
    for words in items:
        if words == "wearing":
            continue
        elif words == "wear":
            continue
        elif words == "person":
            continue
        else:
            temporar.append(words)
    liste.append(temporar)

In [12]:
listc=  []

for items in listb:
    captured = " ".join(items)
    listc.append(captured)

In [13]:
df["questionsAsked"] = listc

## Creating the model of Logistic Regression

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["questionsAsked"],df["hair"],random_state = 0)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer().fit(X_train)

In [17]:
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<15831x1148 sparse matrix of type '<type 'numpy.int64'>'
	with 34607 stored elements in Compressed Sparse Row format>

## Presenting results

There are much more information about the results in the thesis document (doc file)

In [18]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_vectorized, y_train)

from sklearn.metrics import roc_auc_score

predictions = model.predict(vect.transform(X_test))

print (roc_auc_score(y_test,predictions))


0.993630220109


## Coefficients

The below table presents the 20 largest and the 20 smallest coefficients which were captured from the word "hair"

In [19]:
import numpy as np

feature_names = np.array(vect.get_feature_names())

sorted_coef_index = model.coef_[0].argsort()

print ("Smallest coefs:\n{}\n".format(feature_names[sorted_coef_index[:20]]))
print ("Largest coefs:\n{}\n".format(feature_names[sorted_coef_index[:1-20:-1]]))


Smallest coefs:
[u'nan' u'boy' u'girl' u'glasses' u'wearing' u'bangs' u'shirt' u'beard'
 u'asian' u'curley' u'white' u'beautiful' u'fashioned' u'male' u'looks'
 u'head' u'female' u'xxx' u'tall' u'lot']

Largest coefs:
[u'hair' u'haircut' u'hairs' u'haired' u'hairstyle' u'hairline' u'facial'
 u'strange' u'longhair' u'sitting' u'chair' u'receding' u'receeding'
 u'alternative' u'tied' u'less' u'somewhere' u'whether']

